In [1]:
!pwd

/home/giakhang/dev/pose_sandbox


In [2]:
import os
import sys
import glob

sys.path.append("/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D/arm_and_hand")
sys.path.append("/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D")

from dataloader_ann import HandArmLandmarksDataset_ANN

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from datetime import datetime
import pandas as pd

from utilities import fuse_landmarks_from_two_cameras, convert_to_left_shoulder_coord
from functools import partial
from ann import ANN

import open3d as o3d
from utilities import convert_to_left_shoulder_coord
import time
import joblib

from csv_writer import columns_to_normalize, fusion_csv_columns_name
from landmarks_scaler import LandmarksScaler

from left_arm_angle_calculator import LeftArmAngleCalculator
from left_hand_angle_calculator import LeftHandAngleCalculator
from sklearn.preprocessing import MinMaxScaler
import xgboost
from sklearn.multioutput import MultiOutputRegressor

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
arm_hand_fused_names = ["left shoulder", "left elbow", "left hip", "right shoulder", "right hip", 
 "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP", "INDEX_FINGER_MCP", 
 "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", 
 "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", 
 "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", 
 "PINKY_DIP", "PINKY_TIP", "right elbow"]

In [4]:
left_arm_angle_calculator = LeftArmAngleCalculator(
    num_chain=3, 
    landmark_dictionary=arm_hand_fused_names)
left_hand_angle_calculator = LeftHandAngleCalculator(
    num_chain=2, 
    landmark_dictionary=arm_hand_fused_names)

In [5]:
MODEL_NAME = "xgboost"
DATETIME = "{}".format(datetime.now().strftime("%Y%m%d-%H%M"))
DATE = "{}".format(datetime.now().strftime("%Y%m%d"))
BASE_DIR = "/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D/arm_and_hand/runs/{}".format(MODEL_NAME)
SAVE_DIR = os.path.join(BASE_DIR, DATE, DATETIME)
DATA_DIR = "/home/giakhang/dev/pose_sandbox/data"  

os.makedirs(SAVE_DIR)

SELECTED_DATE = "2024-*"
train_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "train")))
val_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "val")))
body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], 
                  [5, 10], [5, 22], [10, 14], 
                  [14, 18], [18, 22], [6, 7], 
                  [7, 8], [8, 9], [10, 11], 
                  [11, 12], [12, 13], [14, 15], 
                  [15, 16], [16, 17], [18, 19], 
                  [19, 20], [20, 21], [22, 23], 
                  [23, 24], [24, 25]]
train_body_distance_thres = 550
train_leftarm_distance_thres = 550
train_lefthand_distance_thres = 200
val_body_distance_thres=450,
val_leftarm_distance_thres=450,
val_lefthand_distance_thres=150,

In [6]:
# Load the true dataset to get the scaler then pass the scaler to the true and fake dataset
minmax_scaler = MinMaxScaler()
train_dataset = HandArmLandmarksDataset_ANN(train_paths, 
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True)
minmax_scaler.fit_transform(train_dataset._inputs)
scaler_save_path = os.path.join(SAVE_DIR, "input_scaler.pkl")
joblib.dump(minmax_scaler, scaler_save_path)

['/home/giakhang/dev/pose_sandbox/Hand_pose_estimation_3D/arm_and_hand/runs/xgboost/20241017/20241017-1443/input_scaler.pkl']

In [7]:
scaler = LandmarksScaler(columns_to_scale=columns_to_normalize,
    scaler_path=scaler_save_path)
train_dataset = HandArmLandmarksDataset_ANN(train_paths, 
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    scaler=scaler,
    cvt_normalized_xy_to_XY=True)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataset = HandArmLandmarksDataset_ANN(val_paths,
    body_lines,
    lefthand_lines,
    val_body_distance_thres,
    val_leftarm_distance_thres,
    val_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    scaler=scaler,
    cvt_normalized_xy_to_XY=True)
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=True)  

In [8]:
input_data = train_dataset._inputs
input_data.shape

(10278, 288)

In [9]:
ground_truth_data = train_dataset._outputs
ground_truth_data = ground_truth_data.reshape(-1, 3, 48)
ground_truth_data = np.transpose(ground_truth_data, (0, 2, 1))
ground_truth_data.shape

(10278, 48, 3)

In [10]:
n_estimators = 100
max_depth = 5
eta = 0.1
subsample = 1.0
colsample_bytree = 0.3

xg_reg = xgboost.XGBRegressor(objective="reg:squarederror",
                              n_estimators=n_estimators,
                              max_depth=max_depth,
                              eta=eta,
                              subsample=subsample,
                              colsample_bytree=colsample_bytree,
                              device="cuda",
                              eval_metric="mae")
multi_xg_reg = MultiOutputRegressor(xg_reg)

In [11]:
multi_xg_reg.fit(train_dataset._inputs, train_dataset._outputs)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.3, device='cuda',
                                            early_stopping_rounds=None,
                                            enable_categorical=False, eta=0.1,
                                            eval_metric='mae',
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None, ...))

In [12]:
predictions = multi_xg_reg.predict(train_dataset._inputs)

/home/giakhang/py_venv/mmpose/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [14:44:46] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [13]:
pred_output = predictions.reshape(-1, 3, 48)
pred_output = np.transpose(pred_output, (0, 2, 1))
pred_output.shape

(10278, 48, 3)

In [14]:
gt_output = train_dataset._outputs
gt_output = gt_output.reshape(-1, 3, 48)
gt_output = np.transpose(gt_output, (0, 2, 1))
gt_output.shape

(10278, 48, 3)

In [15]:
pred_output = pred_output[:, :26, :]
print(pred_output.shape)

(10278, 26, 3)


In [16]:
gt_output = gt_output[:, :26, :]
print(gt_output.shape)

(10278, 26, 3)


In [17]:
pred_angles = []

for i in range(pred_output.shape[0]):
    output = pred_output[i]
    train_pred_XYZ_wrt_shoulder, xyz_origin = convert_to_left_shoulder_coord(
        output,
        arm_hand_fused_names)
    
    left_arm_result = left_arm_angle_calculator(train_pred_XYZ_wrt_shoulder, 
        parent_coordinate=xyz_origin)
    left_arm_angles = left_arm_result["left_arm"]["angles"]
    left_arm_rot_mats_wrt_origin = left_arm_result["left_arm"]["rot_mats_wrt_origin"]
    last_coordinate_from_left_arm = left_arm_rot_mats_wrt_origin[-1]
    left_hand_result = left_hand_angle_calculator(train_pred_XYZ_wrt_shoulder, 
        parent_coordinate=last_coordinate_from_left_arm)
    
    left_hand_angles = []
    for finger_name in left_hand_angle_calculator.fingers_name:
        finger_i_angles = left_hand_result[finger_name]["angles"].copy()

        # In robot, its finger joint 1 is our finger joint 2, and vice versa (EXCEPT FOR THUMB FINGER). 
        # So that, we need to swap these values.
        if finger_name != "THUMB":
            finger_i_angles[0], finger_i_angles[1] = finger_i_angles[1], finger_i_angles[0]

        left_hand_angles.extend(finger_i_angles)
    
    angles = left_arm_angles.copy()
    angles.extend(left_hand_angles)    
    pred_angles.append(angles)

pred_angles = np.array(pred_angles)

In [18]:
gt_angles = []

for i in range(gt_output.shape[0]):
    gt = gt_output[i]
    train_gt_XYZ_wrt_shoulder, xyz_origin = convert_to_left_shoulder_coord(
        gt,
        arm_hand_fused_names)
    
    left_arm_result = left_arm_angle_calculator(train_gt_XYZ_wrt_shoulder, 
        parent_coordinate=xyz_origin)
    left_arm_angles = left_arm_result["left_arm"]["angles"]
    left_arm_rot_mats_wrt_origin = left_arm_result["left_arm"]["rot_mats_wrt_origin"]
    last_coordinate_from_left_arm = left_arm_rot_mats_wrt_origin[-1]
    left_hand_result = left_hand_angle_calculator(train_gt_XYZ_wrt_shoulder, 
        parent_coordinate=last_coordinate_from_left_arm)
    
    left_hand_angles = []
    for finger_name in left_hand_angle_calculator.fingers_name:
        finger_i_angles = left_hand_result[finger_name]["angles"].copy()

        # In robot, its finger joint 1 is our finger joint 2, and vice versa (EXCEPT FOR THUMB FINGER). 
        # So that, we need to swap these values.
        if finger_name != "THUMB":
            finger_i_angles[0], finger_i_angles[1] = finger_i_angles[1], finger_i_angles[0]

        left_hand_angles.extend(finger_i_angles)
    
    angles = left_arm_angles.copy()
    angles.extend(left_hand_angles)    
    gt_angles.append(angles)

gt_angles = np.array(gt_angles)

In [19]:
angles_errors = np.mean(np.abs(gt_angles - pred_angles), axis=0)

In [20]:
angles_names = ["left_arm_joint_01",
                "left_arm_joint_02",
                "left_arm_joint_03",
                "left_arm_joint_04",
                "left_arm_joint_05",
                "left_arm_joint_06",
                "left_THUMB_joint_01",
                "left_THUMB_joint_02",
                "left_THUMB_joint_03",
                "left_INDEX_joint_01",
                "left_INDEX_joint_02",
                "left_INDEX_joint_03",
                "left_MIDDLE_joint_01",
                "left_MIDDLE_joint_02",
                "left_MIDDLE_joint_03",
                "left_RING_joint_01",
                "left_RING_joint_02",
                "left_RING_joint_03",
                "left_PINKY_joint_01",
                "left_PINKY_joint_02",
                "left_PINKY_joint_03"]

In [21]:
assert len(angles_names) == len(angles_errors)

for joint_error, joint_name in zip(angles_errors, angles_names):
    print(f"{joint_name} mean error: {round(joint_error, 2)}")

left_arm_joint_01 mean error: 5.2
left_arm_joint_02 mean error: 2.69
left_arm_joint_03 mean error: 15.56
left_arm_joint_04 mean error: 5.18
left_arm_joint_05 mean error: 48.86
left_arm_joint_06 mean error: 14.77
left_THUMB_joint_01 mean error: 27.23
left_THUMB_joint_02 mean error: 18.11
left_THUMB_joint_03 mean error: 27.43
left_INDEX_joint_01 mean error: 17.77
left_INDEX_joint_02 mean error: 21.97
left_INDEX_joint_03 mean error: 31.49
left_MIDDLE_joint_01 mean error: 17.7
left_MIDDLE_joint_02 mean error: 23.49
left_MIDDLE_joint_03 mean error: 33.08
left_RING_joint_01 mean error: 18.0
left_RING_joint_02 mean error: 24.32
left_RING_joint_03 mean error: 32.59
left_PINKY_joint_01 mean error: 18.51
left_PINKY_joint_02 mean error: 27.09
left_PINKY_joint_03 mean error: 34.69


# Val set

In [22]:
val_input = val_dataset._inputs
val_input.shape

(2379, 288)

In [23]:
val_gt = val_dataset._outputs
val_gt = val_gt.reshape(-1, 3, 48)
val_gt = np.transpose(val_gt, (0, 2, 1))
val_gt.shape

(2379, 48, 3)

In [25]:
val_preds = multi_xg_reg.predict(val_input)
val_preds = val_preds.reshape(-1, 3, 48)
val_preds = np.transpose(val_preds, (0, 2, 1))
val_preds.shape

(2379, 48, 3)

In [26]:
val_preds = val_preds[:, :26, :]
print(val_preds.shape)

(2379, 26, 3)


In [27]:
val_gt = val_gt[:, :26, :]
print(val_gt.shape)

(2379, 26, 3)


In [28]:
pred_angles = []

for i in range(val_preds.shape[0]):
    val_pred = val_preds[i]
    val_pred_XYZ_wrt_shoulder, xyz_origin = convert_to_left_shoulder_coord(
        val_pred,
        arm_hand_fused_names)
    
    left_arm_result = left_arm_angle_calculator(val_pred_XYZ_wrt_shoulder, 
        parent_coordinate=xyz_origin)
    left_arm_angles = left_arm_result["left_arm"]["angles"]
    left_arm_rot_mats_wrt_origin = left_arm_result["left_arm"]["rot_mats_wrt_origin"]
    last_coordinate_from_left_arm = left_arm_rot_mats_wrt_origin[-1]
    left_hand_result = left_hand_angle_calculator(val_pred_XYZ_wrt_shoulder, 
        parent_coordinate=last_coordinate_from_left_arm)
    
    left_hand_angles = []
    for finger_name in left_hand_angle_calculator.fingers_name:
        finger_i_angles = left_hand_result[finger_name]["angles"].copy()

        # In robot, its finger joint 1 is our finger joint 2, and vice versa (EXCEPT FOR THUMB FINGER). 
        # So that, we need to swap these values.
        if finger_name != "THUMB":
            finger_i_angles[0], finger_i_angles[1] = finger_i_angles[1], finger_i_angles[0]

        left_hand_angles.extend(finger_i_angles)
    
    angles = left_arm_angles.copy()
    angles.extend(left_hand_angles)    
    pred_angles.append(angles)

pred_angles = np.array(pred_angles)

In [29]:
gt_angles = []

for i in range(val_gt.shape[0]):
    gt = val_gt[i]
    val_gt_XYZ_wrt_shoulder, xyz_origin = convert_to_left_shoulder_coord(
        gt,
        arm_hand_fused_names)
    
    left_arm_result = left_arm_angle_calculator(val_gt_XYZ_wrt_shoulder, 
        parent_coordinate=xyz_origin)
    left_arm_angles = left_arm_result["left_arm"]["angles"]
    left_arm_rot_mats_wrt_origin = left_arm_result["left_arm"]["rot_mats_wrt_origin"]
    last_coordinate_from_left_arm = left_arm_rot_mats_wrt_origin[-1]
    left_hand_result = left_hand_angle_calculator(val_gt_XYZ_wrt_shoulder, 
        parent_coordinate=last_coordinate_from_left_arm)
    
    left_hand_angles = []
    for finger_name in left_hand_angle_calculator.fingers_name:
        finger_i_angles = left_hand_result[finger_name]["angles"].copy()

        # In robot, its finger joint 1 is our finger joint 2, and vice versa (EXCEPT FOR THUMB FINGER). 
        # So that, we need to swap these values.
        if finger_name != "THUMB":
            finger_i_angles[0], finger_i_angles[1] = finger_i_angles[1], finger_i_angles[0]

        left_hand_angles.extend(finger_i_angles)
    
    angles = left_arm_angles.copy()
    angles.extend(left_hand_angles)    
    gt_angles.append(angles)

gt_angles = np.array(gt_angles)

In [30]:
angles_errors = np.mean(np.abs(gt_angles - pred_angles), axis=0)

In [31]:
angles_names = ["left_arm_joint_01",
                "left_arm_joint_02",
                "left_arm_joint_03",
                "left_arm_joint_04",
                "left_arm_joint_05",
                "left_arm_joint_06",
                "left_THUMB_joint_01",
                "left_THUMB_joint_02",
                "left_THUMB_joint_03",
                "left_INDEX_joint_01",
                "left_INDEX_joint_02",
                "left_INDEX_joint_03",
                "left_MIDDLE_joint_01",
                "left_MIDDLE_joint_02",
                "left_MIDDLE_joint_03",
                "left_RING_joint_01",
                "left_RING_joint_02",
                "left_RING_joint_03",
                "left_PINKY_joint_01",
                "left_PINKY_joint_02",
                "left_PINKY_joint_03"]

In [32]:
assert len(angles_names) == len(angles_errors)

for joint_error, joint_name in zip(angles_errors, angles_names):
    print(f"{joint_name} mean error: {round(joint_error, 2)}")

left_arm_joint_01 mean error: 8.89
left_arm_joint_02 mean error: 5.6
left_arm_joint_03 mean error: 31.03
left_arm_joint_04 mean error: 10.02
left_arm_joint_05 mean error: 88.34
left_arm_joint_06 mean error: 24.03
left_THUMB_joint_01 mean error: 45.09
left_THUMB_joint_02 mean error: 23.0
left_THUMB_joint_03 mean error: 38.02
left_INDEX_joint_01 mean error: 22.73
left_INDEX_joint_02 mean error: 32.22
left_INDEX_joint_03 mean error: 38.69
left_MIDDLE_joint_01 mean error: 21.7
left_MIDDLE_joint_02 mean error: 33.42
left_MIDDLE_joint_03 mean error: 41.13
left_RING_joint_01 mean error: 22.6
left_RING_joint_02 mean error: 35.05
left_RING_joint_03 mean error: 40.36
left_PINKY_joint_01 mean error: 23.34
left_PINKY_joint_02 mean error: 37.61
left_PINKY_joint_03 mean error: 39.13
